# Olson (2D) vs Ray3D (3D) Velocity

Dieses Notebook zeigt zwei Geschwindigkeits-Berechnungen für ein Augenpaar:
- Olson 2D (Positionsdifferenz im Screen, ggf. in °/s umgerechnet)
- Ray3D (Winkeländerung zwischen Blickrichtungs-Vektoren)

Wir nutzen Beispielwerte und rechnen beide Varianten durch.

In [ ]:
import numpy as np


def olson_velocity_deg_per_s(p0_px, p1_px, dt_ms, pixel_pitch_mm=0.264, viewing_dist_mm=600.0):
    """
    Olson 2D: Positionsdifferenz auf dem Screen -> Winkel -> °/s.
    p0_px, p1_px: (x,y) in Pixeln
    dt_ms: Zeitdiff in Millisekunden
    pixel_pitch_mm: mm pro Pixel (Displayabhängig)
    viewing_dist_mm: Beobachterabstand
    """
    p0 = np.array(p0_px, dtype=float)
    p1 = np.array(p1_px, dtype=float)
    dist_px = np.linalg.norm(p1 - p0)
    dist_mm = dist_px * pixel_pitch_mm
    theta_deg = np.degrees(np.arctan2(dist_mm, viewing_dist_mm))
    return theta_deg / (dt_ms / 1000.0)


def ray3d_velocity_deg_per_s(r0, r1, dt_ms):
    """
    Ray3D: Winkel zwischen normierten Blickrichtungs-Vektoren -> °/s.
    r0, r1: (x,y,z) Richtungsvektoren
    dt_ms: Zeitdiff in Millisekunden
    """
    r0 = np.array(r0, dtype=float)
    r1 = np.array(r1, dtype=float)
    r0 /= np.linalg.norm(r0)
    r1 /= np.linalg.norm(r1)
    dot = np.clip(np.dot(r0, r1), -1.0, 1.0)
    theta_deg = np.degrees(np.arccos(dot))
    return theta_deg / (dt_ms / 1000.0)


In [ ]:
# Beispielwerte
p0 = (100, 200)
p1 = (130, 212)
dt_ms = 8.33  # ~120 Hz
r0 = (0.10, 0.00, 0.995)
r1 = (0.15, 0.00, 0.989)

v_olson = olson_velocity_deg_per_s(p0, p1, dt_ms)
v_ray3d = ray3d_velocity_deg_per_s(r0, r1, dt_ms)

print(f"Olson 2D velocity: {v_olson:.2f} °/s")
print(f"Ray3D velocity:    {v_ray3d:.2f} °/s")


**Formeln:**

- Olson (2D): \( v = \frac{\arctan(\frac{\|p_t - p_{t-\Delta t}\|_{mm}}{d_{view}})}{\Delta t}\) in °/s, mit Pixel→mm.
- Ray3D: \( v = \frac{\arccos(\text{clip}(r_t \cdot r_{t-\Delta t}))}{\Delta t} \) in °/s (r sind normierte Blickrichtungen).

In diesem Beispiel zeigen die Zahlen, dass Ray3D die reine Winkeländerung misst, während Olson über die projizierte Positionsänderung geht.